In [1]:
import numpy as np
import tensorflow as tf
import os
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
train_path = "/home/mtq/cikm2017/jupyter/data/datauint8.pkl"
train_x, train_y = pickle.load(open(train_path, 'rb'))
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.2, random_state=1120)
train_x = np.asarray(train_x, dtype=np.uint8)
test_x = np.asarray(test_x, dtype=np.uint8)
train_y = np.asarray(train_y, dtype=np.float64)
test_y = np.asarray(test_y, dtype=np.float64)

In [3]:
online_path = "/home/mtq/cikm2017/jupyter/data/onlinedata.pkl"
online_x= pickle.load(open(online_path, 'rb'))
online_x = np.asarray(online_x, dtype=np.uint8)
online_x = online_x.reshape((-1, 60, 101, 101))

In [4]:
from sklearn.utils import shuffle
train_x, train_y = shuffle(train_x, train_y, random_state=0)
test_x, test_y = shuffle(test_x, test_y, random_state=0)
train_x = train_x.reshape((-1, 60, 101, 101))
test_x = test_x.reshape((-1, 60, 101, 101))
train_y = train_y.reshape((-1, 1))
test_y = test_y.reshape((-1, 1))

In [5]:
import scipy.misc
# Resize image
# misc.imresize(train_x[i], (112, 112))
temp_list = []
for i in range(len(train_x)):
    for j in range(len(train_x[0])):
        temp_list.append(scipy.misc.imresize(train_x[i][j], (112, 112)))
train_x = np.asarray(temp_list)

temp_list = []
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        temp_list.append(scipy.misc.imresize(test_x[i][j], (112, 112)))
test_x = np.asarray(temp_list)

In [6]:
temp_list = []
for i in range(len(online_x)):
    for j in range(len(online_x[0])):
        temp_list.append(scipy.misc.imresize(online_x[i][j], (112, 112)))
online_x = np.asarray(temp_list)
online_x = online_x.reshape((-1, 4, 15, 112, 112))
online_x = np.transpose(online_x, axes=[0, 3, 4, 1, 2])

In [7]:
train_x = train_x.reshape((-1, 4, 15, 112, 112))
test_x = test_x.reshape((-1, 4, 15, 112, 112))
train_x = np.transpose(train_x, axes=[0, 3, 4, 1, 2])
test_x = np.transpose(test_x, axes=[0, 3, 4, 1, 2])

In [8]:
result_path = "/home/mtq/DerHua/wechat/results/lstm15_cnn5x5_fc2.txt"

In [9]:
def Write_to_wechat(last_result):
    f = open(result_path, 'r+')
    result = f.readline()
    result = f.read()
    result = result + last_result + "\n"
    f.seek(0, 0)
    f.truncate()
    f.write(result)
    f.close()


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(bottom, weight, padding):
    return tf.nn.conv2d(bottom, weight, strides=[1, 1, 1, 1], padding=padding)


def conv_layer(bottom, shape, filter, padding='SAME'):
    W = weight_variable(shape)
    biases = bias_variable(filter)
    conv = conv2d(bottom, W, padding)
    return tf.nn.relu(conv + biases)


def max_pool(bottom):
    return tf.nn.max_pool(bottom, ksize=[1, 4, 4, 1], strides=[1, 4, 4, 1], padding='SAME')


def fc_layer(bottom, shape, size):
    W = weight_variable(shape)
    biases = bias_variable(size)
    return tf.matmul(bottom, W) + biases


def lstm_layer(bottom, n_inputs, n_hidden_units, n_output, batch_size):
#     bottom = tf.reshape(bottom, [-1, n_inputs])     # (batch*101*101*4, 15)
#     bottom_in = fc_layer(bottom, [n_inputs, n_hidden_units], [n_hidden_units])  # (batch*101*101*4, 64)
    bottom_in = tf.reshape(bottom, [-1, 1, n_hidden_units])  # (batch*101*101*4, 1, 15)

    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units, forget_bias=1.0, state_is_tuple=True)
    _init_state = lstm_cell.zero_state(batch_size*112*112*4, dtype=tf.float32)
    outputs, states = tf.nn.dynamic_rnn(lstm_cell, bottom_in, initial_state=_init_state, time_major=False)
    outputs = tf.unstack(tf.transpose(outputs, [1, 0, 2]))  # outputs[-1]:(1, batch*101*101*4, 15)
    outputs = fc_layer(outputs[-1], [n_hidden_units, n_output], [n_output])
    outputs = tf.reshape(outputs, [-1, 112, 112, 4])
    return outputs


def build_vgg(bottom, batch_size, keep_prob=1):
    """
    :param: shape of (batch, 101, 101, 4, 15) remember to scale the value
    :return: VGG model
    """
    lstm_bottom = lstm_layer(bottom, 15, 15, 1, batch_size)

    conv1_1 = conv_layer(lstm_bottom, [3, 3, 4, 64], [64])  # 112 * 112
    conv1_2 = conv_layer(conv1_1, [3, 3, 64, 64], [64])  # 112 * 112
    pool1 = max_pool(conv1_2)  # 28 * 28

    conv2_1 = conv_layer(pool1, [3, 3, 64, 128], [128])
    conv2_2 = conv_layer(conv2_1, [3, 3, 128, 128], [128])
    conv2_3 = conv_layer(conv2_2, [3, 3, 128, 128], [128])
    pool2 = max_pool(conv2_3)  # 7 * 7

#     conv3_1 = conv_layer(pool2, [3, 3, 128, 256], [256])  # 28 *28
#     conv3_2 = conv_layer(conv3_1, [3, 3, 256, 256], [256])
#     conv3_3 = conv_layer(conv3_2, [3, 3, 256, 256], [256])
#     pool3 = max_pool(conv3_3)  # 14 * 14

#     conv4_1 = conv_layer(pool3, [3, 3, 256, 256], [256])
#     conv4_2 = conv_layer(conv4_1, [3, 3, 256, 256], [256])
#     conv4_3 = conv_layer(conv4_2, [3, 3, 256, 256], [256])
#     pool4 = max_pool(conv4_3)  # 7 * 7

    conv_flat = tf.reshape(pool2, [-1, 7*7*128])
    fc6 = fc_layer(conv_flat, [7*7*128, 1024], [1024])
    relu6 = tf.nn.relu(fc6)
    relu6_drop = tf.nn.dropout(relu6, keep_prob)

    fc7 = fc_layer(relu6_drop, [1024, 1024], [1024])
    relu7 = tf.nn.relu(fc7)
    relu7_drop = tf.nn.dropout(relu7, keep_prob)

    fc8 = fc_layer(relu7_drop, [1024, 1], [1])

#     fc9 = fc_layer(fc8, fc9_shape, [fc9_out_size])

    return fc8


def culculate_loss(value, prediction):
    return tf.reduce_sum(tf.square(tf.subtract(value, prediction)))

In [10]:
with tf.device("/gpu:5"):
    x = tf.placeholder(tf.float32, [None, 112, 112, 4, 15])
    y = tf.placeholder(tf.float32, [None, 1])
    b = tf.placeholder(tf.int32)
    prediction = build_vgg(x, b, keep_prob=0.4)
    loss = culculate_loss(y, prediction)
    optimizer = tf.train.AdamOptimizer(0.0001)
    train_step = optimizer.minimize(loss)

In [11]:
import sys

train_size = 40
test_size = 40
online_size = 40


def get_batch_train(x, y, i, train_size):
    return x[i * train_size: (i + 1) * train_size], y[i * train_size: (i + 1) * train_size]


def get_batch_test(x, y, i, test_size):
    return x[i * test_size: (i + 1) * test_size], y[i * test_size: (i + 1) * test_size]

def get_batch_online(x, i, online_size):
    return x[i * online_size: (i + 1) * online_size]

In [12]:
total_train_iteration = len(train_x) / train_size
total_iteration = len(test_x) / test_size
total_online_iteration = len(online_x) / online_size

config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [13]:
# saver = tf.train.Saver()
# saver.restore(sess, "/home/mtq/cikm2017/jupyter/DerHua/my_model/lstm_vgg12_regression/-30")
sess.run(tf.global_variables_initializer())
for i in range(0, 4000):
    if i % 10 == 0:
        saver = tf.train.Saver()
        save_path = saver.save(sess, "/home/mtq/cikm2017/jupyter/DerHua/my_model/lstm15_cnn5x5_fc2/", global_step=i)

    total_l2 = 0.0
    total_l1 = 0.0
    online_y = []
    
    for online_batch_step in range(int(total_online_iteration)):
        online_batch_x = get_batch_online(online_x, online_batch_step, online_size)
        online_y.append(prediction.eval(feed_dict={x: online_batch_x, b: online_size}, session=sess))
    online_y_mean = np.mean(online_y)
    online_y = np.asarray(online_y)
    print("step: " + str(i) + ", online mean: " + str(online_y_mean))
    online_y[online_y < 0] = 0
    csv_path = "/home/mtq/cikm2017/jupyter/DerHua/results/lstm15_cnn5x5_fc2/" + "step" + str(i) + "_" + str(online_y_mean) + ".csv"
    online_y= pd.DataFrame(data={'Result': online_y.flatten()})
    online_y.to_csv(csv_path, index=False, header=False)
    online_y.tail()
        
    print("\nstart evaluate")
    for test_batch_step in range(int(total_iteration)):
        test_batch_x, test_batch_y = get_batch_test(test_x, test_y, test_batch_step, test_size)
        total_l2 += loss.eval(feed_dict={
            x: test_batch_x, y: test_batch_y, b: test_size}, session=sess)

    str_result = "step: " + str(i) + ", test rsem: " + str(np.sqrt(total_l2 / float(len(test_x))))
    print(str_result)
    Write_to_wechat(str_result)

    for train_batch_step in range(int(total_train_iteration)):
        if train_batch_step % 40 == 0:
            str_log = ("train: %.2f %% " % (train_batch_step * 100 / total_train_iteration))
            sys.stdout.write('\r' + str_log)
        batch_x, batch_y = get_batch_train(train_x, train_y, train_batch_step, train_size)
        sess.run(train_step, feed_dict={x: batch_x, y: batch_y, b: train_size})
    for train_batch_step in range(int(total_train_iteration)):
        if train_batch_step % 40 == 0:
            str_log = ("train loss: %.2f %% " % (train_batch_step * 100 / total_train_iteration))
            sys.stdout.write('\r' + str_log)
        batch_x, batch_y = get_batch_train(train_x, train_y, train_batch_step, train_size)
        total_l1 += loss.eval(feed_dict={x: batch_x, y: batch_y, b: train_size}, session=sess)
    train_result = "step: " + str(i) + ", train rsem: " + str(np.sqrt(total_l1 / float(len(train_x))))
    print('\r' + train_result)
    Write_to_wechat(train_result)


step: 0, online mean: -81.5796

start evaluate
step: 0, test rsem: 201.374302296
step: 0, train rsem: 21.6478396649
step: 1, online mean: 16.0068

start evaluate
step: 1, test rsem: 22.388873454
step: 1, train rsem: 18.0392762259
step: 2, online mean: 16.5384

start evaluate
step: 2, test rsem: 18.7519744794
step: 2, train rsem: 16.9040088824
step: 3, online mean: 14.1889

start evaluate
step: 3, test rsem: 17.4860496294
step: 3, train rsem: 16.4419480173
step: 4, online mean: 15.3027

start evaluate
step: 4, test rsem: 16.9937710397
step: 4, train rsem: 16.1401949948
step: 5, online mean: 15.0998

start evaluate
step: 5, test rsem: 16.7488994216
step: 5, train rsem: 16.1725245152
step: 6, online mean: 15.9429

start evaluate
step: 6, test rsem: 16.7232755484
step: 6, train rsem: 16.0527643712
step: 7, online mean: 14.4951

start evaluate
step: 7, test rsem: 16.8100479523
step: 7, train rsem: 15.8972564634
step: 8, online mean: 14.7893

start evaluate
step: 8, test rsem: 16.6243329797


KeyboardInterrupt: 